Equations de départ

$$\frac{\delta u}{\delta t}=D_u\nabla^2u-uv^2+F(1-u)$$
$$\frac{\delta v}{\delta t}=D_v\nabla^2v+uv^2-(F+k)v$$

Equation discrétisée

$\frac{u^{n+1}_{i,j}-u^{n}_{i,j}}{\Delta t}=D_u(\frac{u^{n+1}_{i+1,j}-2u^{n+1}_{i,j}+u^{n+1}_{i-1,j}}{\Delta x^2}+\frac{u^{n+1}_{i,j+1}-2u^{n+1}_{i,j}+u^{n+1}_{i,j-1}}{\Delta y^2})-u^{n}_{i,j}(v^{n}_{i,j})^2+F(1-u^{n}_{i,j})
\\
\frac{v^{n+1}_{i,j}-v^{n}_{i,j}}{\Delta t}=D_v(\frac{v^{n+1}_{i+1,j}-2v^{n+1}_{i,j}+v^{n+1}_{i-1,j}}{\Delta x^2}+\frac{v^{n+1}_{i,j+1}-2v^{n+1}_{i,j}+v^{n+1}_{i,j-1}}{\Delta y^2})+u^{n}_{i,j}(v^{n}_{i,j})^2-(F+k)v^{n}_{i,j}$

On met ce qu'on ne connait pas a gauche et ce qu'on connait a droite et en utilisant le fait que $\Delta x=\Delta y=\delta$

$-u^{n+1}_{i-1,j}-u^{n+1}_{i+1,j}+(\frac{\delta^2}{D_u\Delta t}+4)u^{n+1}_{i,j}-u^{n+1}_{i,j-1}-u^{n+1}_{i,j+1}=\frac{\delta^2}{D_u\Delta t}(u^{n}_{i,j}-u^{n}_{i,j}(v^{n}_{i,j})^2+F(1-u^{n}_{i,j}))
\\
-v^{n+1}_{i-1,j}-v^{n+1}_{i+1,j}+(\frac{\delta^2}{D_v\Delta t}+4)v^{n+1}_{i,j}-v^{n+1}_{i,j-1}-v^{n+1}_{i,j+1}=\frac{\delta^2}{D_v\Delta t}(v^{n}_{i,j}+u^{n}_{i,j}(v^{n}_{i,j})^2-(F+k)v^{n}_{i,j}$

In [4]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

Conditions initiales

In [2]:
n = 192
Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 
dh = 5./(n-1)
T = 8000
dt = .9 * dh**2 / (4*max(Du,Dv))
nt = int(T/dt)

On importe le fichier téléchargé, avec les conditions initiales pour U et V

In [3]:
uvinitial = numpy.load('uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']

In [81]:
def constructMatrix(ny, nx, sigma):
    """ Generate implicit matrix for 2D heat equation with Dirichlet in bottom and right and Neumann in top and left
        Assumes dx = dy
    
    Parameters:
    ----------
    nx   : int
        number of discretization points in x
    ny   : int
        number of discretization points in y
    sigma: float
        alpha*dt/dx
        
    Returns:
    -------
    A: 2D array of floats
        Matrix of implicit 2D heat equation
    """
    A = numpy.zeros(((n-2)**2,(n-2)**2))
    
    row_number = 0 # row counter
    for j in range(1,ny-1):
        for i in range(1,nx-1):
            
            # Corners
            if i==1 and j==1: # Bottom left corner (Dirichlet down and left)
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                
            elif i==nx-2 and j==1: # Bottom right corner (Dirichlet down, Neumann right)
                A[row_number,row_number] = 1/sigma+3 # Set diagonal
                A[row_number,row_number-1] = -1      # Fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                
            elif i==1 and j==ny-2: # Top left corner (Neumann up, Dirichlet left)
                A[row_number,row_number] = 1/sigma+3   # Set diagonal
                A[row_number,row_number+1] = -1        # fetch i+1
                A[row_number,row_number-(nx-2)] = -1   # fetch j-1
                
            elif i==nx-2 and j==ny-2: # Top right corner (Neumann up and right)
                A[row_number,row_number] = 1/sigma+2   # Set diagonal
                A[row_number,row_number-1] = -1        # Fetch i-1
                A[row_number,row_number-(nx-2)] = -1   # fetch j-1
              
            # Sides
            elif i==1: # Left boundary (Dirichlet)
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
            
            elif i==nx-2: # Right boundary (Neumann)
                A[row_number,row_number] = 1/sigma+3 # Set diagonal
                A[row_number,row_number-1] = -1      # Fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
                
            elif j==1: # Bottom boundary (Dirichlet)
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number-1] = -1      # fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                
            elif j==ny-2: # Top boundary (Neumann)
                A[row_number,row_number] = 1/sigma+3 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number-1] = -1      # fetch i-1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
                
            # Interior points
            else:
                A[row_number,row_number] = 1/sigma+4 # Set diagonal
                A[row_number,row_number+1] = -1      # fetch i+1
                A[row_number,row_number-1] = -1      # fetch i-1
                A[row_number,row_number+nx-2] = -1   # fetch j+1
                A[row_number,row_number-(nx-2)] = -1 # fetch j-1
                
            row_number += 1 # Jump to next row of the matrix!
    
    return A                    

In [82]:
constructMatrix(n, n, 4)

MemoryError: 

In [78]:
n

192